In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import matplotlib

nd = pd.read_csv('numeric_data.csv')
print(nd.shape)

output = ["stais1","stais3","stais4","stais5","stais6","stais7","stais8","stais9","stais10","stais11","stais12","stais13",
          "stais14","stais15","stais16","stais17","stais18","stais19","stais20",
          "sds_1","sds_3","sds_4","sds_5","sds_6","sds_7","sds_8","sds_9","sds_10","sds_11","sds_12","sds_13",
          "sds_14","sds_15","sds_16","sds_17","sds_18","sds_19","sds_20"]
target = ['stai_s_score', 'sds_score', 'stai_t_score', 'ucls_5', 'sh_score']

data_y = nd.loc[:,output]
data_x = nd

columns = nd.columns.values

for col in columns:
    if col in output:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif col in target:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].isna().sum() != 0:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].dtypes == object:
        data_x = data_x.drop([col],axis = 1)
        #print(nd[col].dtypes)

    '''
    elif not nd[col].dtypes is np.dtype('int64'):
        if not nd[col].dtypes is np.dtype('float64'):
            data_x = data_x.drop([col],axis = 1)
            print(nd[col].dtypes)
    ''' 

#data_y = nd.loc[:,["stai_s_score"]]

features_names = data_x.columns.values
#print(len(features_names))
print(features_names)

copy_data_x = copy.deepcopy(data_x)

data_x = np.array(data_x, dtype=np.float32)
#print(data_x)
data_y = np.array(data_y, dtype=np.float32)
#print(data_y)

#print(features_names)

X = data_x[:1316]
y = data_y[:1316]
X_test = data_x[1316:]
y_test = data_y[1316:]

#print(data_x.shape)
#print(data_y.shape)
#print(data_t.shape)

#print(X.shape)
#print(y.shape)
#print(X_test.shape)
#print(y_test.shape)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


x1 = np.delete(X,slice(263),0)
x2 = np.delete(X,slice(263,526),0)
x3 = np.delete(X,slice(526,790),0)
x4 = np.delete(X,slice(790,1053),0)
x5 = X[:1053]


y1 = np.delete(y,slice(263),0)
y2 = np.delete(y,slice(263,526),0)
y3 = np.delete(y,slice(526,790),0)
y4 = np.delete(y,slice(790,1053),0)
y5 = y[:1053]

X_groups = [x1,x2,x3,x4,x5]
y_groups = [y1,y2,y3,y4,y5]
X_vali = [X[:263],X[263:526],X[526:790],X[790:1053],X[1053:]]
y_vali = [y[:263],y[263:526],y[526:790],y[790:1053],y[1053:]]


class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


input_size = X.shape[1]
hidden_size = 100
output_size = y.shape[1]

learning_rate = 0.01
num_epochs = 100


# Training
mse = 0
for i in range(5):
    X_tensor = torch.tensor(X_groups[i])
    y_tensor = torch.tensor(y_groups[i])
    
    X_validation = torch.tensor(X_vali[i])
    y_validation = torch.tensor(y_vali[i])
    
    
    model = TwoLayerNN(input_size, hidden_size, output_size)

    
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    

    for epoch in range(num_epochs):
        
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
    
    predictions = model(X_validation)
    loss = criterion(predictions, y_validation)
    print("Validation loss:",str(loss.item()))
    mse += loss.item()

CV_k = mse / 5
print(CV_k)

In [ ]:
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)


model = TwoLayerNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)


with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)

print(predictions)
print(f"Test Loss: {test_loss.item()}")


In [ ]:
def stais_score(data):
    return [sum(((5-data[0]),(data[1]),(data[2]),(5-data[3]),(data[4]),(data[5]),(5-data[6]),(data[7]),
               (5-data[8]),(5-data[9]),(data[10]),(data[11]),(data[12]),(5-data[13]),(5-data[14]),
               (data[15]),(data[16]),(5-data[17]),(5-data[18])))#,
            #sum(((data[19]),(data[20]),(data[21]),(5-data[22]),(5-data[23]),(data[24]),(data[25]),(data[26]),
               #(data[27]),(5-data[28]),(5-data[29]),(data[30]),(5-data[31]),(data[32]),(5-data[33]),
               #(5-data[34]),(5-data[35]),(data[36]),(5-data[37])))
           ]

def sds_score(data):
    return [sum(((data[19]),(data[20]),(data[21]),(5-data[22]),(5-data[23]),(data[24]),(data[25]),(data[26]),
               (data[27]),(5-data[28]),(5-data[29]),(data[30]),(5-data[31]),(data[32]),(5-data[33]),
               (5-data[34]),(5-data[35]),(data[36]),(5-data[37])))
           ]

print(1)
#print([int(i+0.5) for i in predictions[0]])
#print(y_test[0].shape)
'''
predict = predictions.tolist()
#print(predict)

p_scores = [scores(i) for i in predict]
y_scores = [scores(i) for i in y_test]
#print(p_scores)

p_sc = [[int(i[0]+0.5),int(i[1]+0.5)] for i in p_scores]
y_sc = [[int(j[0]),int(j[1])] for j in y_scores]
#print(p_sc)
#print()
#print(y_sc)


p_s = torch.tensor(p_scores)
y_s = torch.tensor(y_scores)


l = criterion(p_s, y_s)
print("\nMSEloss:")
print(l.item())
'''

In [ ]:
data_dictionary = pd.read_csv("t1_data_dictionary.csv")
data_dictionary["Field Type"].unique()

data_dictionary.drop_duplicates(subset="Field Type")


categorical_df = data_dictionary[data_dictionary["Field Type"].isin(["radio", "dropdown", "checkbox", "slider"])]
categorical_cols = set(categorical_df["Variable / Field Name"].tolist())
#print(categorical_cols)

#categorical_indices = [i for i in range(len(features_names)) if features_names[i] in categorical_cols]
categorical_indices = []
for i in range(len(features_names)):
    for j in categorical_cols:
        if j in features_names[i]:
            categorical_indices.append(i)
print(categorical_indices)

print([features_names[i] for i in range(len(features_names)) if i in categorical_indices])

In [ ]:
#lime
from lime import lime_tabular

def predict_f1(data, model = model):
    X_tensor = torch.FloatTensor(data)
    predictions = model(X_tensor)
    predict = np.array(predictions.tolist(), dtype=np.float32)
    #print(predict)
    p_scores = np.array([stais_score(i) for i in predict])
    return p_scores

def predict_f2(data, model = model):
    X_tensor = torch.FloatTensor(data)
    predictions = model(X_tensor)
    predict = np.array(predictions.tolist(), dtype=np.float32)
    #print(predict)
    p_scores = np.array([sds_score(i) for i in predict])
    return p_scores

testD = X_test[:10]
#yscores = [scores(i) for i in y_test[:10]]

p1 = predict_f1(data = testD, model = model)
print(p1)

p2 = predict_f2(data = testD, model = model)
print(p2)

In [ ]:
targets = ["stai_s_score"]#, 'sds_score']

explainer = lime_tabular.LimeTabularExplainer(training_data = X, 
                                                   mode = "regression",
                                                   feature_names = features_names,
                                                   categorical_features = categorical_indices,
                                                   class_names = targets, 
                                                   discretize_continuous = True)


importance = {}
count = {}

for j in range(1):
    for j in range(X_test.shape[0]):
        exp = explainer.explain_instance(data_row = X_test[j], predict_fn = predict_f1, num_features = X_test.shape[1])
        exp.show_in_notebook(show_table = True)
        a = exp.as_list()
        #print(a)
        features = []
        
        for i in range(len(a)):
            keys0 = a[i][0].split(' ')
            if ("=" in keys0[0]):
                keys = keys0[0].split("=")
                keys = keys[0].split("__")
                key = keys[0]
            
            else: 
                keys = keys0
                if keys[0][0].isalpha():
                    key = keys[0]
                else:
                    key = keys[2]
            
            if key in importance:
                importance[key] += abs(a[i][1])
                count[key] += 1
            else:
                importance[key] = abs(a[i][1])
                count[key] = 1
            
            features.append(key)
            
        print(features)

print(importance)


In [ ]:
print(X_test.shape[0])
print(sorted(importance.items(), key = lambda kv:(-kv[1], kv[0]))[:50])
print(sorted(count.items(), key = lambda kv:(-kv[1], kv[0]))[:50])

In [ ]:
import matplotlib.pyplot as plt

sortimportance = {}
for f, c in importance.items():
    sortimportance[f] = importance[f] / count[f]

sortimportance = sorted(sortimportance.items(), key = lambda kv:(-abs(kv[1]), kv[0]))[:30]

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(c)
categories.reverse()
frequencies.reverse()

bar_color = []
temp = {}
color_count = 0

for c in categories:
    c = c.split(' ')
    if ("__" in c[0]):
        c = c[0].split("__")[0]
    else:
        if c[0][0].isalpha():
            c = c[0]
        else:
            c = c[2]

    if c in temp:
        bar_color.append(temp[c])
    else:
        temp[c] = list(matplotlib.colors.XKCD_COLORS.values())[color_count]
        color_count += 1
        bar_color.append(temp[c])


plt.figure(figsize=(20, 15))
plt.barh(categories, frequencies, color=bar_color)


plt.title('Bar Chart of Average_Importance by Category')
plt.ylabel('Categories')
plt.xlabel('Average_Importance')


plt.show()

In [ ]:
import matplotlib.pyplot as plt

sortimportance = {}
for f, c in importance.items():
    sortimportance[f] = importance[f] / count[f]

sortimportance = sorted(sortimportance.items(), key = lambda kv:(-abs(kv[1]), kv[0]))[:30]

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(c)
categories.reverse()
frequencies.reverse()

bar_color = []
temp = {}
color_count = 0

for c in categories:
    c = c.split(' ')
    if ("__" in c[0]):
        c = c[0].split("__")[0]
    else:
        if c[0][0].isalpha():
            c = c[0]
        else:
            c = c[2]

    if c in temp:
        bar_color.append(temp[c])
    else:
        temp[c] = list(matplotlib.colors.XKCD_COLORS.values())[color_count]
        color_count += 1
        bar_color.append(temp[c])


plt.figure(figsize=(20, 15))
plt.barh(categories, frequencies, color=bar_color)


plt.title('Bar Chart of Average_Importance by Category')
plt.ylabel('Categories')
plt.xlabel('Average_Importance')


plt.show()

In [ ]:
import matplotlib.pyplot as plt
sortimportance = sorted(importance.items(), key = lambda kv:(-kv[1], kv[0]))
#sortcount = sorted(count.items(), key = lambda kv:(-kv[1], kv[0]))

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(importance[f] / count[f])
    #print(f, importance[f], count[f])
categories.reverse()
frequencies.reverse()


plt.figure(figsize=(20, 20))
plt.barh(categories, frequencies, color='lightblue')


plt.title('Bar Chart of Frequencies by Category')
plt.ylabel('Categories')
plt.xlabel('Frequencies')


plt.show()

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(training_data = X, 
                                                   mode = "regression",
                                                   feature_names = features_names,
                                                   categorical_features = categorical_indices,
                                                   #class_names = targets, 
                                                   discretize_continuous = True)


importance2 = {}
count2 = {}

for i in range(1):
    for j in range(X_test.shape[0]):
        exp = explainer.explain_instance(data_row = X_test[j], predict_fn = predict_f1, num_features = X_test.shape[1])
        exp.show_in_notebook(show_table = True)
        a = exp.as_list()
        #print(a)
        features = []
        
        for i in range(len(a)):
            keys = a[i][0]
            
            if keys in importance2:
                importance2[keys] += a[i][1]
                count2[keys] += 1
            else:
                importance2[keys] = a[i][1]
                count2[keys] = 1
            
            features.append(keys)
            
        print(features)

print(importance2)

In [ ]:
print(X_test.shape[0])
print(sorted(importance2.items(), key = lambda kv:(-abs(kv[1]), kv[0]))[:50])
print(sorted(count2.items(), key = lambda kv:(-kv[1], kv[0]))[:50])

In [ ]:
import matplotlib.pyplot as plt

sortimportance = sorted(importance2.items(), key = lambda kv:(-abs(kv[1]), kv[0]))
#sortcount = sorted(count.items(), key = lambda kv:(-kv[1], kv[0]))

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(importance2[f] / count2[f])
    #print(f, importance[f], count[f])
categories.reverse()
frequencies.reverse()


bar_color = []
temp = {}
color_count = 0

for c in categories:
    c = c.split(' ')
    if ("=" in c[0]):
        c = c[0].split("=")[0].split("__")[0]
    else:
        if c[0][0].isalpha():
            c = c[0]
        else:
            c = c[2]

    if c in temp:
        bar_color.append(temp[c])
    else:
        temp[c] = list(matplotlib.colors.XKCD_COLORS.values())[color_count] #matplotlib.colors.Colormap(color_count)
        color_count += 1
        bar_color.append(temp[c])

#print(bar_color)


plt.figure(figsize=(20, 30))
plt.barh(categories, frequencies, color= bar_color)


plt.title('Bar Chart of Frequencies by Category')
plt.ylabel('Categories')
plt.xlabel('Frequencies')


plt.show()

In [ ]:
import matplotlib.pyplot as plt

sortimportance = {}
for f, c in importance2.items():
    sortimportance[f] = importance2[f] / count2[f]

sortimportance = sorted(sortimportance.items(), key = lambda kv:(-abs(kv[1]), kv[0]))[:50]
#sortcount = sorted(count.items(), key = lambda kv:(-kv[1], kv[0]))

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(c)
categories.reverse()
frequencies.reverse()

bar_color = []
temp = {}
color_count = 0

for c in categories:
    c = c.split(' ')
    if ("=" in c[0]):
        c = c[0].split("=")[0].split("__")[0]
    else:
        if c[0][0].isalpha():
            c = c[0]
        else:
            c = c[2]

    if c in temp:
        bar_color.append(temp[c])
    else:
        temp[c] = list(matplotlib.colors.XKCD_COLORS.values())[color_count]
        color_count += 1
        bar_color.append(temp[c])

#print(bar_color)


plt.figure(figsize=(20, 10))
plt.barh(categories, frequencies, color=bar_color)


plt.title('Bar Chart of Frequencies by Category')
plt.ylabel('Categories')
plt.xlabel('Frequencies')


plt.show()

In [ ]:
import matplotlib.pyplot as plt

sortimportance = {}
for f, c in importance2.items():
    sortimportance[f] = importance2[f] / count2[f]

sortimportance = sorted(sortimportance.items(), key = lambda kv:(-abs(kv[1]), kv[0]))
#sortcount = sorted(count.items(), key = lambda kv:(-kv[1], kv[0]))

categories, frequencies = [], []
for f, c in sortimportance:
    categories.append(f)
    frequencies.append(c)
categories.reverse()
frequencies.reverse()

bar_color = []
temp = {}
color_count = 0

for c in categories:
    c = c.split(' ')
    if ("=" in c[0]):
        c = c[0].split("=")[0].split("__")[0]
    else:
        if c[0][0].isalpha():
            c = c[0]
        else:
            c = c[2]

    if c in temp:
        bar_color.append(temp[c])
    else:
        temp[c] = list(matplotlib.colors.XKCD_COLORS.values())[color_count]
        color_count += 1
        bar_color.append(temp[c])

#print(bar_color)


plt.figure(figsize=(20, 20))
plt.barh(categories, frequencies, color=bar_color)


plt.title('Bar Chart of Frequencies by Category')
plt.ylabel('Categories')
plt.xlabel('Frequencies')


plt.show()